## TensorFlow implementation of Convolution Neural Network


In [20]:
import math
import numpy as np
import tensorflow as tf
from tensorflow.python.framework import ops
%matplotlib inline
np.random.seed(0)

In [21]:
def PlaceHolders(n_H0, n_W0, n_C0, n_y):
    """
    Arguments:
    n_H0 -- scalar, height of an input image
    n_W0 -- scalar, width of an input image
    n_C0 -- scalar, number of channels of the input
    n_y -- scalar, number of classes
        
    Returns:
    X -- placeholder for the data input, of shape [None, n_H0, n_W0, n_C0] and dtype "float"
    Y -- placeholder for the input labels, of shape [None, n_y] and dtype "float"
    """
    X = tf.placeholder(dtype='float', shape=[None, n_H0, n_W0, n_C0])
    Y = tf.placeholder(dtype='float', shape=[None, n_y])

    
    return X, Y

In [22]:
X, Y = PlaceHolders(128, 128, 3, 10)
print ("X = " + str(X))
print ("Y = " + str(Y))

X = Tensor("Placeholder_2:0", shape=(?, 128, 128, 3), dtype=float32)
Y = Tensor("Placeholder_3:0", shape=(?, 10), dtype=float32)


In [23]:
def initialize_parameters():
    """
    Returns:
    parameters -- a dictionary of tensors containing W1, W2
    """
    
    tf.set_random_seed(0)   
    #Always use get_variables() to initialize variables as it gives you more power over the variables
    W1 =  tf.get_variable("W1", [4,4,3,8], initializer = tf.contrib.layers.xavier_initializer(seed = 0))
    W2 =  tf.get_variable("W2", [2,2,8,16], initializer = tf.contrib.layers.xavier_initializer(seed = 0))

    parameters = {"W1": W1,
                  "W2": W2}
    
    return parameters

In [29]:
tf.reset_default_graph() # Resets the default graph
with tf.Session() as sess:
    parameters = initialize_parameters()
    init = tf.global_variables_initializer()
    sess.run(init)
    print("W1 = " + str(parameters["W1"].eval()[1,1,1]))
    print("W2 = " + str(parameters["W2"].eval()[1,1,1]))

W1 = [ 0.10274635 -0.15470493 -0.13962929  0.07496916 -0.09490568  0.11148815
 -0.07839259  0.13499068]
W2 = [-0.07568306  0.19660437 -0.13395107  0.01072127  0.15392095  0.18631959
 -0.17422909 -0.0297823   0.0661872  -0.05305719  0.15413994 -0.06312037
 -0.12061065 -0.18098605 -0.23810583  0.00187701]


<b>TensorFlow provides methods that can be used directly to implement Convolution Layers, Pooling layers and Fully connected layers. Read the TensorFlow documentation for more details.</b> [TensorFlow](https://www.tensorflow.org/api_docs/python/tf/contrib)

In [25]:
def forward_propagation(X, parameters):
    """
    CONV2D -> RELU -> MAXPOOL -> CONV2D -> RELU -> MAXPOOL -> FLATTEN -> FULLYCONNECTED
    
    Arguments:
    X -- input dataset placeholder, of shape (input size, number of examples)
    parameters -- python dictionary containing your parameters "W1", "W2"
                  the shapes are given in initialize_parameters

    Returns:
    Z3 -- the output of the last LINEAR unit
    """
    
    # Retrieve the parameters from the dictionary "parameters" 
    W1 = parameters['W1']
    W2 = parameters['W2']
    
    
    # CONV2D: stride of 1, padding 'SAME'
    Z1 = tf.nn.conv2d(X,W1, strides = [1,1,1,1], padding = 'SAME')
    # RELU
    A1 = tf.nn.relu(Z1)
    # MAXPOOL: window 8x8, sride 8, padding 'SAME'
    P1 = tf.nn.max_pool(A1, ksize = [1,8,8,1], strides = [1,8,8,1], padding = 'SAME')
    # CONV2D: filters W2, stride 1, padding 'SAME'
    Z2 = tf.nn.conv2d(P1,W2, strides = [1,1,1,1], padding = 'SAME')
    # RELU
    A2 = tf.nn.relu(Z2)
    # MAXPOOL: window 4x4, stride 4, padding 'SAME'
    P2 = tf.nn.max_pool(A2, ksize = [1,4,4,1], strides = [1,4,4,1], padding = 'SAME')
    # FLATTEN
    P2 = tf.contrib.layers.flatten(P2)
    # FULLY-CONNECTED without non-linear activation function.
    Z3 = tf.contrib.layers.fully_connected(P2, 6, activation_fn=None)
    
    return Z3

In [26]:
tf.reset_default_graph()

with tf.Session() as sess:
    np.random.seed(0)
    X, Y = PlaceHolders(64, 64, 3, 6)
    parameters = initialize_parameters()
    Z3 = forward_propagation(X, parameters)
    init = tf.global_variables_initializer()
    sess.run(init)
    a = sess.run(Z3, {X: np.random.randn(2,64,64,3), Y: np.random.randn(2,6)})
    print("Z3 = " + str(a))

Z3 = [[ -2.98667955e+00   4.35540080e-03   1.84640288e+00  -7.05335569e+00
    5.35991430e-01   2.82552195e+00]
 [ -2.73371482e+00   4.93455678e-01   2.01306319e+00  -6.99065971e+00
    3.39891225e-01   2.87566876e+00]]


In [27]:
def compute_cost(Z3, Y):
    """
    Computes the cost
    
    Arguments:
    Z3 -- output of forward propagation (output of the last LINEAR unit)
    Y -- "true" labels vector placeholder, same shape as Z3
    
    Returns:
    cost - Tensor of the cost function
    """
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = Z3, labels = Y))
    
    return cost

In [28]:
tf.reset_default_graph()

with tf.Session() as sess:
    np.random.seed(0)
    X, Y = PlaceHolders(64, 64, 3, 6)
    parameters = initialize_parameters()
    Z3 = forward_propagation(X, parameters)
    cost = compute_cost(Z3, Y)
    init = tf.global_variables_initializer()
    sess.run(init)
    a = sess.run(cost, {X: np.random.randn(4,64,64,3), Y: np.random.randn(4,6)})
    print("cost = " + str(a))

cost = 6.6979


## How to build a Model using the above implemented helper functions

Finally you will merge the helper functions you implemented above to build a model. You will train it on the SIGNS dataset. 


you can create the Model that does the following- by using the helper functions implemented above:

- create placeholders
- initialize parameters
- forward propagate
- compute the cost
- create an optimizer

Finally you will create a session and run a for loop  for num_epochs, get the mini-batches, and then for each mini-batch you will optimize the function. [How to Initialize Variables](https://www.tensorflow.org/api_docs/python/tf/global_variables_initializer)

<b> I will later implement the model and add a new cell containing model to this source code. </b>